<a href="https://colab.research.google.com/github/diegosantos9/maps/blob/main/OK_MAPA_Gyn_e_Ap_Gyn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# SCRIPT CONSOLIDADO V2: LEILÃO DE IMÓVEIS (GOIÂNIA E APARECIDA DE GOIÂNIA)
# Extrai, geocodifica e mapeia imóveis com tratamento de erros e paginação dinâmica.
# ==============================================================================

# ------------------------------------------------------------------------------
# 1. CONFIGURAÇÃO E INSTALAÇÃO DE BIBLIOTECAS
# ------------------------------------------------------------------------------
print("📦 Instalando bibliotecas necessárias...")
!pip install -q cloudscraper beautifulsoup4 pandas openpyxl geopy folium tqdm
print("✅ Bibliotecas instaladas.")

import re
import time
import pandas as pd
import folium
import cloudscraper
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm.auto import tqdm

# ------------------------------------------------------------------------------
# 2. EXTRAÇÃO DE DADOS (SCRAPING) COM PAGINAÇÃO DINÂMICA
# ------------------------------------------------------------------------------
print("\n🔄 Iniciando a extração de dados do site Leilão Imóvel...")

# Dicionário com as cidades e seus respectivos códigos no site
CIDADES = [
    {'nome': 'Goiânia', 'codigo': '5208707', 'cor': '#007bff'}, # Azul
    {'nome': 'Aparecida de Goiânia', 'codigo': '5201405', 'cor': '#dc3545'} # Vermelho
]

# Inicializa o scraper que simula um navegador para evitar bloqueios
scraper = cloudscraper.create_scraper(
    browser={'browser': 'chrome', 'platform': 'windows', 'mobile': False}
)

def clean_text(text):
    """Remove caracteres de controle ilegais que causam o erro ao salvar em Excel."""
    # Regex para encontrar caracteres de controle XML inválidos
    return re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1F]', '', str(text))

def parse_card(card, cidade_nome):
    """Extrai informações de um card de imóvel, valida e limpa os dados."""
    txt = " ".join(card.get_text(" ", strip=True).split())

    if txt == "+" or len(txt) < 10:
        return None

    link = "https://www.leilaoimovel.com.br" + card["href"]

    m_val = re.search(r"R\$ ([\d\.]+,\d{2})", txt)
    valor = m_val.group(1) if m_val else "À Consultar"

    m_desc = re.search(r"(\d{1,3})%", txt)
    desconto = m_desc.group(1) if m_desc else "0"

    tipo = ("Apartamento" if "Apto" in txt else
            "Casa" if "Casa" in txt else
            "Terreno" if "Terreno" in txt else "Outro")

    natureza = ("Judicial" if "Judicial" in txt else
                "Extrajudicial" if "Extrajudicial" in txt else
                "Venda Direta" if "Venda Direta" in txt else "Outro")

    end = re.split(r"[A-Z]{2}\s*-\s*\d+\s*", txt, 1)[-1]
    end = re.split(r"1ª Praça|2ª Praça|Valor|R\$", end, 1)[0].strip(" -")

    if end == "+" or len(end) < 5:
        return None

    # Aplica a limpeza nos campos de texto antes de retornar
    return {
        "cidade": clean_text(cidade_nome),
        "endereco": clean_text(end),
        "valor": valor,
        "desconto_%": desconto,
        "tipo": clean_text(tipo),
        "natureza": clean_text(natureza),
        "link": link
    }

# Loop principal para extrair dados de todas as cidades configuradas
todos_imoveis = []
base_url = "https://www.leilaoimovel.com.br/encontre-seu-imovel"

for cidade in CIDADES:
    cidade_nome = cidade['nome']
    cidade_codigo = cidade['codigo']

    print(f"\n🔎 Buscando imóveis em {cidade_nome}...")

    current_page = 1
    with tqdm(desc=f"Páginas ({cidade_nome})", unit=" pág") as pbar:
        while True:
            url = f"{base_url}?cidade={cidade_codigo}&ordem=dt_insert_d&pag={current_page}&s="
            imoveis_encontrados_na_pagina = 0

            try:
                html = scraper.get(url, timeout=45).text
                soup = BeautifulSoup(html, "html.parser")
                cards = soup.select('a[href^="/imovel/"]')

                if not cards:
                    print(f" Fim dos resultados para {cidade_nome} na página {current_page}.")
                    break # Encerra o loop para esta cidade

                for a in cards:
                    item = parse_card(a, cidade_nome)
                    if item:
                        todos_imoveis.append(item)
                        imoveis_encontrados_na_pagina += 1

                # Se a página veio vazia de itens válidos, também paramos
                if imoveis_encontrados_na_pagina == 0:
                    print(f" Fim dos resultados para {cidade_nome} na página {current_page}.")
                    break

                current_page += 1
                pbar.update(1)
                time.sleep(1)

            except Exception as e:
                print(f" Erro ao acessar a página {current_page} de {cidade_nome}: {e}")
                break # Encerra em caso de erro de conexão, etc.

# Cria um DataFrame único e remove duplicatas
df = pd.DataFrame(todos_imoveis).drop_duplicates(subset=["endereco", "link"])

# Exporta para um único arquivo Excel
ARQUIVO_EXCEL = "imoveis_consolidados.xlsx"
df.to_excel(ARQUIVO_EXCEL, index=False)
print(f"\n✅ Excel consolidado gerado com {len(df)} imóveis: {ARQUIVO_EXCEL}")

# ------------------------------------------------------------------------------
# 3. GEOCODIFICAÇÃO (TRANSFORMAR ENDEREÇO EM COORDENADAS)
# ------------------------------------------------------------------------------
print("\n🌍 Iniciando geocodificação dos endereços...")

def limpar_endereco(row):
    """Limpa o endereço e garante que a cidade e estado estejam presentes."""
    end = str(row['endereco'])
    cidade = row['cidade']

    end = re.sub(r"\s*\+\s*$", "", end)
    end = re.sub(r"\blote\b.*?$", "", end, flags=re.I).strip(" ,–-")

    if cidade not in end and not end.lower().endswith("goiás"):
        end += f", {cidade}, Goiás"

    if "Aparecida" in cidade:
        cep = re.search(r"(\d{5}-\d{3})", end)
        if cep:
            return f"{cep.group(1)}, {cidade}, Goiás"

    return end

df["endereco_geo"] = df.apply(limpar_endereco, axis=1)

geolocator = Nominatim(user_agent="leilao_imoveis_go_consolidado_v2")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.1, error_wait_seconds=10)

def geo_fallback(end, cidade):
    """Tenta geocodificar o endereço com várias estratégias de fallback."""
    loc = geocode(end)
    if loc: return loc

    m_setor = re.search(r"Setor\s+([A-Za-zÀ-ÿ\s]+)", end, re.I)
    if m_setor:
        loc = geocode(f"Setor {m_setor.group(1).strip()}, {cidade}, Goiás")
        if loc: return loc

    m_rua = re.search(r"\b(Rua|Av\.|Avenida)\b[^,]+", end, re.I)
    if m_rua:
        loc = geocode(f"{m_rua.group(0).strip()}, {cidade}, Goiás")
        if loc: return loc

    return geocode(f"Centro, {cidade}, Goiás")

lat, lon = [], []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Geocodificando"):
    location = geo_fallback(row["endereco_geo"], row["cidade"])
    lat.append(location.latitude if location else None)
    lon.append(location.longitude if location else None)

df["lat"], df["lon"] = lat, lon
df_geocodificado = df.dropna(subset=["lat", "lon"])

print(f"\n📊 Total de endereços: {len(df)} | Geocodificados com sucesso: {len(df_geocodificado)}")

# ------------------------------------------------------------------------------
# 4. GERAÇÃO DO MAPA CONSOLIDADO
# ------------------------------------------------------------------------------
print("\n🗺️  Gerando o mapa com a localização dos imóveis...")

mapa_centro = [-16.75, -49.25]
mapa = folium.Map(location=mapa_centro, zoom_start=11, tiles="CartoDB Positron")

def valor_float(v):
    """Converte o valor em string para float para definir o raio do marcador."""
    try:
        return float(str(v).replace(".", "").replace(",", "."))
    except (ValueError, TypeError):
        return 100000

cidade_cor_map = {c['nome']: c['cor'] for c in CIDADES}

for _, row in df_geocodificado.iterrows():
    popup_html = f"""
        <b>Cidade:</b> {row.cidade}<br>
        <b>Endereço:</b> {row.endereco}<br>
        <b>Tipo:</b> {row.tipo}<br>
        <b>Valor:</b> R$ {row.valor}<br>
        <a href="{row.link}" target="_blank"><b>Ver anúncio no site</b></a>
    """

    radius = max(5, min(valor_float(row.valor) / 75000, 20))

    folium.CircleMarker(
        location=[row.lat, row.lon],
        radius=radius,
        color=cidade_cor_map.get(row.cidade, '#333333'),
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=350)
    ).add_to(mapa)

legend_html = '''
     <div style="position: fixed;
     bottom: 50px; left: 50px; width: 180px; height: 90px;
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white; padding: 10px;">
     <b>Legenda</b><br>
     <i class="fa fa-circle" style="color:#007bff"></i>  Goiânia<br>
     <i class="fa fa-circle" style="color:#dc3545"></i>  Aparecida de Goiânia<br>
     <small>(Tamanho do círculo proporcional ao valor)</small>
     </div>
     '''
mapa.get_root().html.add_child(folium.Element(legend_html))

ARQUIVO_MAPA = "mapa_leiloes_goias.html"
mapa.save(ARQUIVO_MAPA)

print("\n\n✅ Processo concluído com sucesso!")
print(f"➡️ Arquivo Excel salvo como: {ARQUIVO_EXCEL}")
print(f"➡️ Mapa interativo salvo como: {ARQUIVO_MAPA}")
print("   (Para visualizar o mapa, faça o download do arquivo HTML e abra-o no seu navegador).")

📦 Instalando bibliotecas necessárias...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.4 MB/s eta 0:00:00
✅ Bibliotecas instaladas.

🔄 Iniciando a extração de dados do site Leilão Imóvel...

🔎 Buscando imóveis em Goiânia...


Páginas (Goiânia): 0 pág [00:00, ? pág/s]

 Fim dos resultados para Goiânia na página 17.

🔎 Buscando imóveis em Aparecida de Goiânia...


Páginas (Aparecida de Goiânia): 0 pág [00:00, ? pág/s]

 Fim dos resultados para Aparecida de Goiânia na página 8.

✅ Excel consolidado gerado com 426 imóveis: imoveis_consolidados.xlsx

🌍 Iniciando geocodificação dos endereços...


Geocodificando:   0%|          | 0/426 [00:00<?, ?it/s]

A saída de streaming foi truncada nas últimas 5000 linhas.
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/http/client.py", line 1395, in getresponse
    response.begin()
  File "/usr/lib/python3.11/http/client.py", line 325, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/http/client.py", line 286, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/


📊 Total de endereços: 426 | Geocodificados com sucesso: 426

🗺️  Gerando o mapa com a localização dos imóveis...


✅ Processo concluído com sucesso!
➡️ Arquivo Excel salvo como: imoveis_consolidados.xlsx
➡️ Mapa interativo salvo como: mapa_leiloes_goias.html
   (Para visualizar o mapa, faça o download do arquivo HTML e abra-o no seu navegador).
